    Aanalysis Data and Clean

In [13]:
import pandas as pd

# ==============================
# LOAD DATASET
# ==============================
df = pd.read_csv(
    "C:/Users/Jitesh Jangam/Desktop/Future Int/Ecommerce/Dataset/Amazon Sale Report.csv"
)

# ==============================
# 1. DROP USELESS / GARBAGE COLUMNS
# ==============================
df.drop(
    columns=[col for col in df.columns if "Unnamed" in col or col.lower() == "index"],
    inplace=True,
    errors="ignore"
)

# ==============================
# 2. STANDARDIZE COLUMN NAMES
# ==============================
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# ==============================
# 3. REMOVE DUPLICATE HEADER ROWS (CRITICAL FIX)
# ==============================
if "date" in df.columns:
    df = df[df["date"].astype(str).str.lower() != "date"]

# ==============================
# 4. FIX DATE COLUMN (SQL-SAFE)
# ==============================
if "date" in df.columns:
    df["date"] = pd.to_datetime(
        df["date"],
        errors="coerce",
        dayfirst=True
    )

    # DROP invalid dates (MANDATORY)
    df.dropna(subset=["date"], inplace=True)

    # Force ISO format for SQL DATE
    df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# ==============================
# 5. CLEAN NUMERIC COLUMNS
# ==============================
numeric_cols = ["amount", "qty"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "")
            .str.strip()
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

# ==============================
# 6. HANDLE MISSING VALUES
# ==============================
critical_cols = ["order_id", "amount"]
df.dropna(subset=[c for c in critical_cols if c in df.columns], inplace=True)

categorical_cols = df.select_dtypes(include="object").columns
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

# ==============================
# 7. FILTER VALID SALES ONLY
# ==============================
if "status" in df.columns:
    df = df[~df["status"].str.lower().isin(["cancelled", "returned"])]

# ==============================
# 8. CREATE DERIVED TIME COLUMNS
# ==============================
if "date" in df.columns:
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["month"] = pd.to_datetime(df["date"]).dt.month
    df["month_name"] = pd.to_datetime(df["date"]).dt.strftime("%B")

if "b2b" in df.columns:
    df["b2b"] = (
        df["b2b"]
        .astype(str)
        .str.lower()
        .map({"true": 1, "false": 0})
    )

df.drop(columns=["promotion-ids"], inplace=True, errors="ignore")
# ==============================
# 9. FINAL VALIDATION
# ==============================
print("Final dataset shape:", df.shape)
print(df.info())
print(df.head())

df.to_csv("amazon_sales_sql_ready.csv", index=False)
# ==============================
# 10. SAVE SQL-READY FILE
# ==============================
print("✅ SQL-ready file saved: amazon_sales_sql_ready.csv")


C:\Users\Jitesh Jangam\AppData\Local\Temp\ipykernel_24808\2799031272.py:6: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Jitesh Jangam\AppData\Local\Temp\ipykernel_24808\2799031272.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(


Final dataset shape: (110414, 24)
<class 'pandas.core.frame.DataFrame'>
Index: 110414 entries, 1 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            110414 non-null  object 
 1   date                110414 non-null  object 
 2   status              110414 non-null  object 
 3   fulfilment          110414 non-null  object 
 4   sales_channel       110414 non-null  object 
 5   ship-service-level  110414 non-null  object 
 6   style               110414 non-null  object 
 7   sku                 110414 non-null  object 
 8   category            110414 non-null  object 
 9   size                110414 non-null  object 
 10  asin                110414 non-null  object 
 11  courier_status      110414 non-null  object 
 12  qty                 110414 non-null  int64  
 13  currency            110414 non-null  object 
 14  amount              110414 non-null  float64
 15  ship-